## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependecies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import weather_api_key

# Configure gmaps API key
gmaps.configure(api_key= weather_api_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/Weather_data/WeatherPy_database.csv")
#city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cockburn Town,TC,21.4612,-71.1419,83.43,74,5,11.14,clear sky
1,1,Rawson,AR,-43.3002,-65.1023,44.40,61,98,14.23,overcast clouds
2,2,Meulaboh,ID,4.1363,96.1285,74.21,87,99,4.41,overcast clouds
3,3,Barra Patuca,HN,15.8000,-84.2833,81.21,80,73,7.61,broken clouds
4,4,Busselton,AU,-33.6500,115.3333,50.94,78,89,6.69,overcast clouds


In [ ]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? (°F) "))
max_temp = float(input("What is the maximum temperature you would like for your trip? (°F) "))

In [ ]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

In [ ]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

In [ ]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_cities_df = preferred_cities_df.dropna()
clean_cities_df

In [ ]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

In [ ]:
# 6a. Set parameters to search for hotels with 5000 meters. Use the search term: "lodging" 
params = {
    "radius": 5000,
    "type": "lodging",
    "key": weather_api_key
}

# 6b. Iterate through the hotel DataFrame.
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
       hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(str(index) + "Hotel not found... skipping.")
        
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")
     

In [ ]:
hotel_df.count()

In [ ]:
hotel_df

In [ ]:
#7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace("", np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna(subset=["Hotel Name"])
clean_hotel_df

In [ ]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig